### This notebook builds on [this work.](https://www.kaggle.com/code/kerta27/mmlm2025-randomforest-optuna-lb-0-00000) It uses the feature engineering done by Kert, but implements a feed-forward neural network in PyTorch

In [16]:
import numpy as np
import pandas as pd
from sklearn import *
#import redisAI
import glob

path = './kaggle/input/march-machine-learning-mania-2025/**'
data = {p.split('/')[-1].split('.')[0].split('\\')[1] : pd.read_csv(p, encoding='latin-1') for p in glob.glob(path)}

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Data Preparation</p>

In [17]:
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
del teams_spelling

In [18]:
season_cresults = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']])
season_dresults = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']])
tourney_cresults = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']])
tourney_dresults = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']])
seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']])
gcities = pd.concat([data['MGameCities'], data['WGameCities']])
seasons = pd.concat([data['MSeasons'], data['WSeasons']])

seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
cities = data['Cities']
sub = data['SampleSubmissionStage2']
del data

In [19]:
season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'
#games = pd.concat((season_cresults, tourney_cresults), axis=0, ignore_index=True)
games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})

games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1)
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed'] 
games = games.fillna(-1)

c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF']
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

games = games[games['ST']=='T']

sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] 
sub = sub.fillna(-1)

games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm', 'WLoc'] + c_score_col]

In [20]:
col

['Season',
 'Team1Seed',
 'Team2Seed',
 'SeedDiff',
 'IDTeams_c_score',
 'NumOTsum_c_score',
 'NumOTmean_c_score',
 'NumOTmedian_c_score',
 'NumOTmax_c_score',
 'NumOTmin_c_score',
 'NumOTstd_c_score',
 'NumOTskew_c_score',
 'NumOTnunique_c_score',
 'WFGMsum_c_score',
 'WFGMmean_c_score',
 'WFGMmedian_c_score',
 'WFGMmax_c_score',
 'WFGMmin_c_score',
 'WFGMstd_c_score',
 'WFGMskew_c_score',
 'WFGMnunique_c_score',
 'WFGAsum_c_score',
 'WFGAmean_c_score',
 'WFGAmedian_c_score',
 'WFGAmax_c_score',
 'WFGAmin_c_score',
 'WFGAstd_c_score',
 'WFGAskew_c_score',
 'WFGAnunique_c_score',
 'WFGM3sum_c_score',
 'WFGM3mean_c_score',
 'WFGM3median_c_score',
 'WFGM3max_c_score',
 'WFGM3min_c_score',
 'WFGM3std_c_score',
 'WFGM3skew_c_score',
 'WFGM3nunique_c_score',
 'WFGA3sum_c_score',
 'WFGA3mean_c_score',
 'WFGA3median_c_score',
 'WFGA3max_c_score',
 'WFGA3min_c_score',
 'WFGA3std_c_score',
 'WFGA3skew_c_score',
 'WFGA3nunique_c_score',
 'WFTMsum_c_score',
 'WFTMmean_c_score',
 'WFTMmedian_c_sco

<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Train Model</p>

In [21]:
import optuna
from sklearn import ensemble
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [22]:
imputer = SimpleImputer(strategy='mean')  
scaler = StandardScaler()

In [31]:
X = games[col].fillna(-1)
X

,Season,Team1Seed,Team2Seed,SeedDiff,IDTeams_c_score,NumOTsum_c_score,NumOTmean_c_score,NumOTmedian_c_score,NumOTmax_c_score,NumOTmin_c_score,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2003,16.0,16.0,0.0,1411_1421,1,1.00,1.0,1,1,...,-1.000000,1,22,22.000000,22.0,22,22,-1.000000,-1.000000,1
1,2003,1.0,16.0,-15.0,1112_1436,0,0.00,0.0,0,0,...,-1.000000,1,15,15.000000,15.0,15,15,-1.000000,-1.000000,1
2,2003,10.0,7.0,3.0,1113_1272,0,0.00,0.0,0,0,...,-1.000000,1,18,18.000000,18.0,18,18,-1.000000,-1.000000,1
3,2003,11.0,6.0,5.0,1141_1166,0,0.00,0.0,0,0,...,-1.000000,2,35,17.500000,17.5,21,14,4.949747,-1.000000,2
4,2003,8.0,9.0,-1.0,1143_1301,1,0.25,0.0,1,0,...,-0.422521,4,66,16.500000,17.0,19,13,2.645751,-0.863919,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,2024,3.0,1.0,2.0,3163_3425,0,0.00,0.0,0,0,...,-1.000000,2,32,16.000000,16.0,20,12,5.656854,-1.000000,2
2272,2024,1.0,3.0,-2.0,3234_3261,0,0.00,0.0,0,0,...,-1.000000,2,40,20.000000,20.0,21,19,1.414214,-1.000000,2
2273,2024,3.0,1.0,2.0,3163_3234,0,0.00,0.0,0,0,...,1.732051,2,35,11.666667,10.0,18,7,5.686241,1.205659,3
2274,2024,3.0,1.0,2.0,3301_3376,0,0.00,0.0,0,0,...,0.190198,7,104,14.857143,15.0,22,8,5.014265,-0.159771,6


In [23]:
X = games[col].fillna(-1)
X_imputed = imputer.fit_transform(X)
X_scaled = scaler.fit_transform(X_imputed)

y = games['Pred']

In [24]:
print(X_scaled.shape, y.shape)
print(X_scaled.dtype, y.dtype)
y

(2276, 221) (2276,)
float64 float64


0       0.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2271    1.0
2272    1.0
2273    0.0
2274    0.0
2275    0.0
Name: Pred, Length: 2276, dtype: float64

In [25]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import random_split

# turn numpy arrays into tensors
X_train_tensor = torch.tensor(X_scaled, dtype=torch.float32)  # Convert features to float
y_train_tensor = torch.tensor(y, dtype=torch.float32)  # Convert labels to long (for classification)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Train, valid split
train_size = int(0.8 * len(train_dataset))  # 80% for training
val_size = len(train_dataset) - train_size  # Remaining 20% for validation
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])


batch_size = 32  # Set batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [26]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, d_in, d_out, d_hidden, n_layers = 2):
        super().__init__()
        layers = [nn.Linear(d_in, d_hidden), nn.BatchNorm1d(d_hidden),
            nn.ReLU()]
        for layer in range(n_layers):
            layers += [nn.Linear(d_hidden, d_hidden), nn.BatchNorm1d(d_hidden),nn.ReLU(), nn.Dropout(p=0.3)]
        layers += [nn.Linear(d_hidden, d_out)]
        self.linear_relu_stack = nn.Sequential(*layers)

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [27]:
import torch.optim as optim

# Define mode, loss function and optimizer

model = NeuralNetwork(X_scaled.shape[1], 1, 100)
print(model)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust learning rate as needed

# Training function
def train(model, train_loader, loss_fn, optimizer, device):
    model.train()  # Set model to training mode
    total_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)  # Move data to GPU if available

        optimizer.zero_grad()  # Reset gradients
        outputs = model(batch_X).squeeze()  # Forward pass (ensure output shape matches labels)
        loss = loss_fn(outputs, batch_y)  # Compute MSE loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        total_loss += loss.item()  # Accumulate loss
    
    return total_loss / len(train_loader)  # Return average loss per batch

# Evaluation function
def evaluate(model, val_loader, loss_fn, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    
    with torch.no_grad():  # Disable gradient computation
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X).squeeze()  # Ensure output shape matches labels
            
            loss = loss_fn(outputs, batch_y)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    return avg_loss  # Return MSE loss (lower is better)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=221, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=100, out_features=100, bias=True)
    (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Dropout(p=0.3, inplace=False)
    (11): Linear(in_features=100, out_features=1, bias=True)
  )
)


In [28]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 5  # Adjust the number of epochs based on performance

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    val_loss = evaluate(model, val_loader, loss_fn, device)
    
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Train Loss (MSE): {train_loss:.4f}")
    print(f"  Validation Loss (MSE): {val_loss:.4f}")

Epoch 1/5:
  Train Loss (MSE): 0.3097
  Validation Loss (MSE): 0.2158
Epoch 2/5:
  Train Loss (MSE): 0.2377
  Validation Loss (MSE): 0.2366
Epoch 3/5:
  Train Loss (MSE): 0.2166
  Validation Loss (MSE): 0.1940
Epoch 4/5:
  Train Loss (MSE): 0.1968
  Validation Loss (MSE): 0.2121
Epoch 5/5:
  Train Loss (MSE): 0.1955
  Validation Loss (MSE): 0.1882


<p style="background-color: rgb(247, 230, 202); font-size: 300%; text-align: center; border-radius: 40px 40px; color: rgb(162, 87, 79); font-weight: bold; font-family: 'Roboto'; border: 4px solid rgb(162, 87, 79);">Submission</p>

In [29]:
X_submit = sub[col].fillna(-1)
X_submit_imputed = imputer.transform(X_submit)
X_submit_scaled = scaler.transform(X_submit_imputed)


X_submit_tensor = torch.tensor(X_submit_scaled, dtype=torch.float32)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_submit_tensor = X_submit_tensor.to(device)

# Set model to evaluation mode
model.eval()

# Make predictions (disable gradients for efficiency)
with torch.no_grad():
    y_preds = model(X_submit_tensor).cpu().numpy()  # Move back to CPU for saving
        

In [30]:
sub['Pred'] = y_preds
sub[['ID', 'Pred']].to_csv('submission.csv', index=False)